# Домашнее задание 4
Условие: используйте источник rate, напишите код, который создаст дополнительный столбец,
который будет выводить сумму только нечётных чисел.

In [75]:
# Зависимости
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum
from pyspark.sql.functions import *


# Создание сессии Spark
spark = SparkSession.builder.appName("Homework_4").getOrCreate()

# Определение потока данных с использованием источника rate
streaming_df = spark.readStream.format("rate").option("rowsPerSecond", 1).load()
print('Выводим изначальный дата фрейм')
query = streaming_df.writeStream.outputMode("update").queryName("streaming_df").format("memory").start()
query.awaitTermination(11)
query.stop()
spark.sql("select distinct * from streaming_df order by value").show(truncate=False)
# Создание пользовательской функции
def chek_value(v):
    if v % 2 != 0:
        return v
    else:
        return 0
chek_value_udf = udf(chek_value, IntegerType())
# Добавление дополнительного столбца с нечетным числом или нулем
odd_numbers_df = streaming_df.withColumn("isOdd", chek_value_udf(streaming_df.value))
print('Выводим дата фрейм с дополнительным столбцом')
query = odd_numbers_df.writeStream.outputMode("update").queryName("odd_values_df").format("memory").start()
query.awaitTermination(11)
query.stop()
spark.sql("select distinct * from odd_values_df order by value").show(truncate=False)
# Сумма нечетных чисел
sum_odd_values_df = odd_numbers_df.groupBy().agg(sum("isOdd").alias('odd_sum'))
print('Выводим суммы по дополнительному столбцу')
query = sum_odd_values_df.writeStream.outputMode("update").queryName("sum_odd_values_df").format("memory").start()
query.awaitTermination(12)
query.stop()
spark.sql("select distinct * from sum_odd_values_df where odd_sum is not NULL order by odd_sum").show(truncate=False)


spark.stop()

Выводим изначальный дата фрейм
+-----------------------+-----+
|timestamp              |value|
+-----------------------+-----+
|2024-12-17 14:58:10.266|0    |
|2024-12-17 14:58:11.266|1    |
|2024-12-17 14:58:12.266|2    |
|2024-12-17 14:58:13.266|3    |
|2024-12-17 14:58:14.266|4    |
|2024-12-17 14:58:15.266|5    |
|2024-12-17 14:58:16.266|6    |
|2024-12-17 14:58:17.266|7    |
|2024-12-17 14:58:18.266|8    |
|2024-12-17 14:58:19.266|9    |
+-----------------------+-----+

Выводим дата фрейм с дополнительным столбцом
+-----------------------+-----+-----+
|timestamp              |value|isOdd|
+-----------------------+-----+-----+
|2024-12-17 14:58:21.592|0    |0    |
|2024-12-17 14:58:22.592|1    |1    |
|2024-12-17 14:58:23.592|2    |0    |
|2024-12-17 14:58:24.592|3    |3    |
|2024-12-17 14:58:25.592|4    |0    |
|2024-12-17 14:58:26.592|5    |5    |
|2024-12-17 14:58:27.592|6    |0    |
|2024-12-17 14:58:28.592|7    |7    |
|2024-12-17 14:58:29.592|8    |0    |
|2024-12-17 14:58:3